In [ ]:

'''
# The month wise labels generated in Exploration are evaluated here.
# 
# Like % of hate, offense, on a month wise basis
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import concurrent.futures
import joblib
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import re
import nltk

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,sent_tokenize
from nltk.stem.porter import *

nltk.download('punkt')

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

from matplotlib import pyplot as plt
from wordcloud import WordCloud
import matplotlib.colors as mcolors

In [ ]:
!pip3 install pickle5

In [ ]:
def remove_emojis(text):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', text)

def remove_hashtags(text):
  return text.replace('#', '').replace('_', ' ')

In [ ]:
def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = remove_hashtags(remove_emojis(parsed_text))
    return parsed_text

In [ ]:
!pip install pickle5
import pickle5 as pickle

with open('drive/MyDrive/dataframes/new_covid_pretrain_predictions.pkl', 'rb') as f:
  df = pickle.load(f)
# df = pd.read_pickle('drive/MyDrive/new_covid_pretrain_predictions.pkl')
df.head(2)

In [ ]:
df.shape

In [ ]:
!ls drive/MyDrive/dataframes/

In [ ]:
import random
indices = random.sample(range(0,df[df.Label == 'Hate'].shape[0]), 25)
df_sample_hate = df[df.Label == 'Hate'].iloc[indices][['TweetID', 'Tweet', 'Label']]
indices = random.sample(range(0,df[df.Label == 'Neutral'].shape[0]), 475)
df_sample_nonhate = df[df.Label == 'Neutral'].iloc[indices][['TweetID', 'Tweet', 'Label']]
df_sample_500 = pd.concat([df_sample_hate, df_sample_nonhate], ignore_index=True)

In [ ]:
df_sample_500 = pd.read_csv('drive/MyDrive/dataframes/df_sample_500.csv', index_col=0)

In [ ]:
import random
indices = random.sample(range(0,df[df.Label == 'Hate'].shape[0]), 500)
df_sample_hate = df[df.Label == 'Hate'].iloc[indices][['TweetID', 'Tweet', 'Label']]
for index, row in df_sample_hate.iterrows():
  if row[0] in list(df_sample_500.TweetID):
    df_sample_hate = df_sample_hate.drop(index)
indices = random.sample(range(0,df_sample_hate[df_sample_hate.Label == 'Hate'].shape[0]), 475)
df_sample_hate = df_sample_hate[df_sample_hate.Label == 'Hate'].iloc[indices][['TweetID', 'Tweet', 'Label']]

indices = random.sample(range(0,df[df.Label == 'Neutral'].shape[0]), 500)
df_sample_nonhate = df[df.Label == 'Neutral'].iloc[indices][['TweetID', 'Tweet', 'Label']]
for index, row in df_sample_nonhate.iterrows():
  if row[0] in list(df_sample_500.TweetID):
    df_sample_nonhate = df_sample_nonhate.drop(index)
indices = random.sample(range(0,df_sample_nonhate[df_sample_nonhate.Label == 'Neutral'].shape[0]), 25)
df_sample_nonhate = df_sample_nonhate[df_sample_nonhate.Label == 'Neutral'].iloc[indices][['TweetID', 'Tweet', 'Label']]

In [ ]:
df_sample_500 = pd.concat([df_sample_hate, df_sample_nonhate], ignore_index=True)
df_sample_500.Label.value_counts()

In [ ]:
df_sample_500 = df_sample_500[['TweetID', 'Tweet', 'Label']].sample(frac=1).reset_index(drop=True)
df_sample_500

In [ ]:
df_sample_500.to_csv('drive/MyDrive/dataframes/df_sample_500_2.csv')

In [ ]:
df_sample_500_task = df_sample_500[['TweetID', 'Tweet']]
df_sample_500_task.to_csv('drive/MyDrive/dataframes/df_sample_500_2_task.csv') 

In [ ]:
df.Label.value_counts()

In [ ]:
all_len = df.shape[0]
print('Whole data: ')
print('Neutral: %.3f' % (df.Label.value_counts()['Neutral']/all_len))
print('Other: %.3f' % (df.Label.value_counts()['Other']/all_len))
print('Hate: %.3f' % (df.Label.value_counts()['Hate']/all_len))
print('Counterhate: %.3f' % (df.Label.value_counts()['Counterhate']/all_len))

# print('\nSample: ')
# print('Counterhate: %.3f' % (df_sample.Label.value_counts()['Neutral']/5000))
# print('Other: %.3f' % (df_sample.Label.value_counts()['Other']/5000))
# print('Hate: %.3f' % (df_sample.Label.value_counts()['Hate']/5000))
# print('Counterhate: %.3f' % (df_sample.Label.value_counts()['Counterhate']/5000))

In [ ]:
df_sample_500['Preprocessed'] = df_sample_500.Tweet.apply(preprocess)
df_sample_500

In [ ]:
df_sample_500 = df_sample_500[['TweetID', 'Preprocessed', 'Label']].sample(frac=1).reset_index(drop=True)
df_sample_500

In [ ]:
# df_sample_500.to_csv('drive/MyDrive/dataframes/df_sample_500.csv')
df_sample_500 = pd.read_csv('drive/MyDrive/dataframes/df_sample_500.csv', index_col=0)

In [ ]:
df_sample_500_anna.Label.value_counts()

In [ ]:
def find_gt(id):
  gt = df_sample_500.Label[df_sample_500.TweetID==id].values[0]
  return 1 if gt == 'Hate' else 0

In [ ]:
df_sample_500_anna = pd.read_csv('drive/MyDrive/dataframes/df_sample_500_task_anna.csv', index_col=0)
df_sample_500_anna.head(2)

In [ ]:
find_gt(1268248943642763264)

In [ ]:
df_sample_500_anna['GT'] = df_sample_500_anna.TweetID.apply(find_gt)

In [ ]:
df_sample_500_anna.GT.value_counts()

In [ ]:
df_sample_500_anna.Label.value_counts()

In [ ]:
def find_corrects(id):
  l = df_sample_500_anna.Label[df_sample_500_anna.TweetID==id].values[0]
  gt = df_sample_500_anna.GT[df_sample_500_anna.TweetID==id].values[0]
  if l*gt ==1:
    print(id)
  return l*gt

In [ ]:
df_sample_500_anna.Label[df_sample_500_anna.TweetID==df_sample_500_anna.iloc[110].TweetID].values[0]

In [ ]:
df_sample_500_anna['Corrects'] = df_sample_500_anna.TweetID.apply(find_corrects)
df_sample_500_anna.head(10)

In [ ]:
df_sample_500_anna.Corrects.value_counts()

In [ ]:
list(df_sample_500_anna[250:].Preprocessed[df_sample_500_anna.GT=='Hate'].values)

In [ ]:
df_sample_500_task = df_sample_500[['TweetID', 'Preprocessed']].sample(frac=1).reset_index(drop=True)
df_sample_500_task.to_csv('drive/MyDrive/dataframes/df_sample_500_task.csv')
df_sample_500_task

In [ ]:
sample_1000 = pd.read_csv('drive/MyDrive/dataframes/df_sample_task_1000.csv', index_col=0)
sample_1000

In [ ]:
manual_labels = list(sample_1000[sample_1000.columns[2]].dropna().astype(int))

In [ ]:
classifications_500_2 = pd.read_csv('drive/MyDrive/dataframes/df_sample_500_2.csv', index_col=0)
classifications_500_2

In [ ]:
classifications = list(classifications_500_2.Label.map(lambda x: 1 if x=='Hate' else 0))

In [ ]:
classifications.count(1)

In [ ]:
manual_labels.count(1)

In [ ]:
len(manual_labels)

In [ ]:
tp, tn, fp, fn = [0,0,0,0]
for i in range(500):
  if classifications[i] == manual_labels[500+i]:
    if classifications[i] == 1:
      tp += 1
    else:
      tn += 1
  else:
    if classifications[i] == 1:
      fp += 1
    else:
      fn += 1
tp, tn, fp, fn

In [ ]:
import inspect
inspect.signature(metrics_calc)

In [ ]:
metrics_calc(tn, fp, fn, tp)

# MTurk HITs

In [ ]:
# create rows with 15 tweets each (except for the last row)
import math
sample_size = 5012
hit_size = 14
rows = []
for i in range(0, int(math.ceil(sample_size/hit_size))):
  row = []
  for j in range(0, hit_size):
    if j+hit_size*i < sample_size:
      tweet_tuple = list(df_selected_columns[['TweetID', 'Preprocessed', 'Label']].values[hit_size*i+j])
      row.extend([str(tweet_tuple[0]), tweet_tuple[1], tweet_tuple[2]])
  rows.append(row)

In [ ]:
df_hits = pd.DataFrame(rows, columns=['id1', 'tweet1', 'label1', 'id2', 'tweet2', 'label2', 'id3', 'tweet3', 'label3', 'id4', 'tweet4', 'label4', 'id5', 'tweet5', 'label5', 'id6', 'tweet6', 'label6', 'id7', 'tweet7', 'label7', 'id8', 'tweet8', 'label8', 'id9', 'tweet9', 'label9', 'id10', 'tweet10', 'label10', 'id11', 'tweet11', 'label11', 'id12', 'tweet12', 'label12', 'id13', 'tweet13', 'label13', 'id14', 'tweet14', 'label14'])
df_hits

In [ ]:
df_hits.to_csv('drive/MyDrive/dataframes/df_hits.csv', float_format=str)

In [ ]:
df_hits_old = pd.read_csv('drive/MyDrive/df_hits.csv', index_col=0, float_precision='high')
df_hits_old.head(2)

In [ ]:
with open('drive/MyDrive/df_hits.csv', 'r') as f:
  lines = f.readlines()[:21]
print(lines[0])
print(lines[1])

In [ ]:
ids = []
for l in lines:
  hit_ids  = []
  for e in l.split(','):
    if len(e) < 6:
      continue
    try:
      hit_ids.append(int(e))
    except:
      pass
  if len(hit_ids) > 0:
    ids.append(hit_ids)

In [ ]:
def find_label(id):
  for index, row in df.iterrows():
    if id == row[0]:
      return row[-1]

In [ ]:
label_list = []
for row in ids:
  for id in row:
    label_list.append([id, df.Label[df.TweetID == id].values[0]])

In [ ]:
label_list

In [ ]:
df_results = pd.read_csv('drive/MyDrive/20hits_result.csv')
def find_answer(df):
  annotations = []
  hit = [[], [], []]
  cnt = 0
  for index, row in df.iterrows():
    if cnt < 3:
      for i in (list(range(55,79,2)) + [81, 83]):
        if row[i] == True:
          hit[cnt].append(1)
        else:
          hit[cnt].append(0)
      cnt += 1
    if cnt == 3:
      total = [hit[0][j]+hit[1][j]+hit[2][j] for j in range(len(hit[0]))]
      annotations.extend([1 if e>1 else 0 for e in total])
      cnt = 0
      hit = [[], [], []]
  return annotations

In [ ]:
majority_labels = find_answer(df_results)

In [ ]:
binary_label_list = [1 if e[1]=='Hate' else 0 for e in label_list]

In [ ]:
np.array(majority_labels)

In [ ]:
np.array(binary_label_list)

In [ ]:
for i in range(len(majority_labels)):
  if majority_labels[i] == 1:
    print(i)

In [ ]:
# with open('drive/MyDrive/Batch_4735221_batch_results_35hits.csv', 'r') as f:
#   lines = f.readlines()
df_35hits = pd.read_csv('drive/MyDrive/dataframes/Batch_4735221_batch_results_35hits.csv')
df_35hits.head(2)

In [ ]:
headers = [
  'Answer.Q1.1', 'Answer.Q2.1', 'Answer.Q3.1', 'Answer.Q4.1', 'Answer.Q5.1', 
  'Answer.Q6.1', 'Answer.Q8.1', 'Answer.Q9.1', 'Answer.Q10.1', 'Answer.Q11.1',
  'Answer.Q12.1', 'Answer.Q13.1', 'Answer.Q14.1', 'Answer.Q15.1'
]

def find_answer_35hits(df):
  annotations = []
  hit = [[], [], []]
  cnt = 0
  for index, row in df.iterrows():
    if cnt < 3:
      for h in headers:
        if row[h] == True:
          hit[cnt].append(1)
        else:
          hit[cnt].append(0)
      cnt += 1
    if cnt == 3:
      total = [hit[0][j]+hit[1][j]+hit[2][j] for j in range(len(hit[0]))]
      annotations.extend([1 if e>2 else 0 for e in total])
      cnt = 0
      hit = [[], [], []]
  return annotations

In [ ]:
anns = find_answer_35hits(df_35hits)

In [ ]:
len(anns)

In [ ]:
sum(anns)

In [ ]:
labels = []
cnt = 0
for index, row in df_35hits.iterrows():
  if cnt == 0:
    for i in list(range(29,69,3)):
      if row[i] == 'Hate':
        labels.append(1)
      elif row[i] == 'Neutral':
        labels.append(0)
      else:
        print('Wrong content!')
  cnt += 1
  if cnt == 3:
    cnt = 0

In [ ]:
tmp = [list(a) for a in df_35hits.iloc[::3, list(range(28,68,3))].values]
tmp = [a[:] for a in tmp]
tweets = []
for i in range(len(tmp)):
  tweets.extend(tmp[i])
tweets

In [ ]:
def metrics_calc(tn, fp, fn, tp):
    cm = [int(tn), int(fp), int(fn), int(tp)]
    recall = 0.0 if (tp+fn == 0) else float(format(tp/(tp+fn), '.4f'))
    tpr = recall
    tnr = 0.0 if (tn+fp == 0) else float(format(tn/(tn+fp), '.4f'))
    fpr = 0.0 if (fp+tn == 0) else float(format(fp/(fp+tn), '.4f'))
    fnr = 0.0 if (fn+tp == 0) else float(format(fn/(fn+tp), '.4f'))
    accuracy = float(format((tp+tn)/(tn+fp+fn+tp), '.4f'))
    precision = 0.0 if (tp+fp == 0) else float(format(tp/(tp+fp), '.4f'))
    f1 = 0.0 if (precision+recall == 0.0) else \
                float(format((2*precision*recall)/(precision+recall), '.4f'))
    print('tn, fp, fn, tp: ', cm)
    print('tpr: ', tpr)
    print('tnr: ', tnr)
    print('fpr: ', fpr)
    print('fnr: ', fnr)
    print('accuracy: ', accuracy)
    print('precision: ', precision)
    print('recall: ', recall)
    print('f1: ', f1)
    return cm, tpr, tnr, fpr, fnr, accuracy, precision, recall, f1

In [ ]:
df_sample_500_anna.Label.value_counts()

In [ ]:
df_sample_500_anna.GT.value_counts()

In [ ]:
metrics_calc(461, 20, 14, 5)

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(anns, labels).ravel()
metrics_calc(tn, fp, fn, tp)

In [ ]:
ground truth: 

490 total tweets
56 hate classified

434 non-hate
5000

In [ ]:
fp = []
fn = []
for i in range(len(anns)):
  if anns[i] != labels[i]:
    if (anns[i]==True) and (labels[i]==False):
      fn.append(tweets[i])
    elif (anns[i]==False) and (labels[i]==True):
      fp.append(tweets[i])

In [ ]:
df_fn = pd.DataFrame(fn, columns=['tweet'])
df_fn['status'] = 'FN'
df_fn

In [ ]:
df_fp = pd.DataFrame(fp, columns=['tweet'])
df_fp['status'] = 'FP'
df_fp

In [ ]:
pd.concat([df_fn, df_fp], ignore_index=True).to_csv('drive/MyDrive/dataframes/wrong-classifications.csv')

In [ ]:
aa=2;bb=5;cc=aa+bb
print(f"{aa} is better than {bb} because of {cc}")

# Processing

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")
other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]+", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]+", tweet.lower())).strip()
    return tweet.split()

In [ ]:

# import nltk
# nltk.download('stopwords')

In [ ]:
#Pre data

df_pre = pd.read_pickle('/content/drive/My Drive/corona_hate_modelling/covid_raw_data/tweet_for_top_2k_pre_event.pkl')
df_pre = df_pre[df_pre['Tweet_Type'] == "tweet"]

df_pre["isTimeString"] = df_pre['TimeStamp'].apply(lambda x : type(x)==type(1))
df_int = df_pre[df_pre["isTimeString"] == True]
df_int['TimeStamp']  = pd.to_datetime(df_int['TimeStamp'],unit='s')

df_str = df_pre[df_pre["isTimeString"] == False]
df_str['TimeStamp'] = pd.to_datetime(df_str['TimeStamp']).values.astype(np.int64) // 10 ** 9
df_str['TimeStamp']  = pd.to_datetime(df_str['TimeStamp'],unit='s')

df_pre = pd.concat([df_int, df_str])
df_pre["Date"] = df_pre["TimeStamp"].apply(lambda x: x.date())

df_pre['month'] = df_pre['TimeStamp'].dt.month
df_pre.groupby('month')['TweetID'].count()


In [ ]:
len(df_pre["UserID"].unique())

In [ ]:
#Post data
df_post = pd.read_pickle('/content/drive/My Drive/corona_hate_modelling/covid_raw_data/tweet_for_top_2k_event.pkl')
df_post = df_post[df_post['Tweet_Type'] == "tweet"]

df_post["isTimeString"] = df_post['TimeStamp'].apply(lambda x : type(x)==type(1))
df_int = df_post[df_post["isTimeString"] == True]
df_int['TimeStamp']  = pd.to_datetime(df_int['TimeStamp'],unit='s')
# df_int.head(2)


df_str = df_post[df_post["isTimeString"] == False]
df_str['TimeStamp'] = pd.to_datetime(df_str['TimeStamp']).values.astype(np.int64) // 10 ** 9
df_str['TimeStamp']  = pd.to_datetime(df_str['TimeStamp'],unit='s')
# df_str.head()

df_post = pd.concat([df_int, df_str])
df_post["Date"] = df_post["TimeStamp"].apply(lambda x: x.date())

df_post['month'] = df_post['TimeStamp'].dt.month
df_post.groupby('month')['TweetID'].count()

In [ ]:
df = pd.concat([df_pre, df_post], axis=0)

len(df["UserID"].unique())

In [ ]:
pre_text = df_pre.Tweet
corona_related_words = set()

for tweet in pre_text:
  p1 = preprocess(tweet)
  p2 = basic_tokenize(p1)

  for wrd in p2:
    if wrd.lower().startswith('coron') or wrd.lower().startswith('viru') or wrd.lower().startswith('covid'):
      corona_related_words.add(wrd)

post_text = df_post.Tweet
for tweet in post_text:
  p1 = preprocess(tweet)
  p2 = basic_tokenize(p1)

  for wrd in p2:
    if wrd.lower().startswith('coron') or wrd.lower().startswith('viru') or wrd.lower().startswith('covid'):
      corona_related_words.add(wrd)

corona_related_words.add('pandemic')

with open('/content/drive/My Drive/corona_hate_modelling/covid_raw_data/corona_words.txt', 'w') as filehandle:
    for listitem in corona_related_words:
        filehandle.write('%s\n' % listitem)




In [ ]:
# ## Function for plotting the word cloud
import matplotlib as mpl
def clusters_whole(df, month, stopwords_arg):
    
    month_tweets = df[df['month'] == month]['Tweet'].to_list()

    clean_month_tweets = []
    for i in month_tweets:
        p1 = preprocess(i)
        p2 = basic_tokenize(p1)
        clean_month_tweets.append(" ".join(p2))


    mpl.rcParams['font.size']=12                #10 
    mpl.rcParams['savefig.dpi']=100             #72 
    mpl.rcParams['figure.subplot.bottom']=.1 


    wordcloud = WordCloud(background_color='white',
                              stopwords=stopwords_arg,
                              max_words=200,
                              max_font_size=40, 
                              random_state=42
                             ).generate(" ".join([i for i in clean_month_tweets]))

#     print(new_corp)
    fig = plt.figure(1)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    
    return wordcloud



In [ ]:
corona_related_words = []

# open file and read the content in a list
with open('/content/drive/My Drive/corona_hate_modelling/covid_raw_data/corona_words.txt', 'r') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        currentPlace = line[:-1]

        # add item to the list
        corona_related_words.append(currentPlace)

In [ ]:
stopwords = list(stopwords) + list(corona_related_words)

stopwords = set(stopwords)

In [ ]:
def get_pred_prec(y_pred):
    
    df = pd.DataFrame()
    df['y_pred'] = y_pred
    
    return (df.groupby('y_pred')['y_pred'].count() / y_pred.shape[0])

In [ ]:
y_pred_dir = '/content/drive/My Drive/corona_hate_modelling/month_wise_data/post_event/'
for month in sorted(df_post['month'].unique()):
    
    print ("### Month ### ", month)
    clusters_whole(df_post, month, stopwords)
    
    
    y_pred = np.load(y_pred_dir + "y_month{}.npy".format(month))
    print (get_pred_prec(y_pred))

In [ ]:
y_pred_dir = '/content/drive/My Drive/corona_hate_modelling/month_wise_data/pre_event/'
for month in sorted(df_pre['month'].unique()):
    
    print ("### Month ### ", month)
    clusters_whole(df_pre, month, stopwords)
    
    
    y_pred = np.load(y_pred_dir + "y_month{}.npy".format(month))
    print (get_pred_prec(y_pred))